In [1]:
import numpy as np
from model import *

In [2]:
config = {}
config['epochs'] = 2
config['temperature'] = 1
config['num_neurons'] = 100
config['num_layers'] = 1
config['input_dim'] = None
config['learning_rate'] = 0.001
config['saved_path'] = 'Saved_weights.pth'


In [ ]:
# Check if your system supports CUDA
use_cuda = torch.cuda.is_available()

# Setup GPU optimization if CUDA is supported
if use_cuda:
    computing_device = torch.device("cuda")
    extras = {"num_workers": 1, "pin_memory": True}
    print("CUDA is supported")
else: # Otherwise, train on the CPU
    computing_device = torch.device("cpu")
    extras = False
    print("CUDA NOT supported")

In [3]:
train_file = open('train.txt', 'r')
train_file = train_file.read()
train_songs = []

val_file = open('val.txt', 'r')
val_file = val_file.read()
val_songs = []

test_file = open('test.txt', 'r')
test_file = test_file.read()
test_songs = []

#split the songs into their own strings, while including start and end tags
for song in train_file.split('<start>')[1:]:
    #remove \n after <end> tag
    train_songs.append('<start>' + song[:-1])
    
for song in val_file.split('<start>')[1:]:
    #remove \n after <end> tag
    val_songs.append('<start>' + song[:-1])

for song in test_file.split('<start>')[1:]:
    #remove \n after <end> tag
    test_songs.append('<start>' + song[:-1])
    
    
unique_characters = list(set(train_file))
config['input_dim'] = len(unique_characters)

#create one hot encodings for each unique character in the alphabet of the training data
one_hot_dict_encode = {}
one_hot_dict_decode = {}
index = 0
for unique_character in unique_characters:
    current_encoding = np.zeros(len(unique_characters))
    current_encoding[index] = 1
    
    one_hot_dict_encode[unique_character] = current_encoding
    one_hot_dict_decode[tuple(current_encoding)] = unique_character
    
    index += 1


print(unique_characters)

[')', '_', 'Y', 'J', 'D', '.', 'j', '0', '1', '*', 'h', '{', 'T', 'K', '(', '~', ',', 'I', '=', 'Z', 'x', 'C', 'X', '#', 'V', '7', 'q', 'N', 'k', '}', '+', 'g', 'y', 'E', 'o', 'n', 'O', '^', 'R', '8', 's', '-', ']', 't', ':', '<', 'H', 'S', 'e', '4', 'B', 'i', '\n', 'L', 'W', 'P', '\\', 'v', 'Q', '"', '2', 'U', '6', '5', 'f', 'w', 'd', 'z', '?', 'A', '[', 'G', 'M', 'F', '\t', 'b', 'l', 'c', '3', '@', "'", '&', '/', '>', '|', '9', '!', 'a', 'u', ' ', 'p', 'r', 'm']


In [4]:
hidden_state = torch.zeros((config['num_layers'], 1, config['input_dim'])).to(computing_device)
cell_state = torch.zeros((config['num_layers'], 1, config['input_dim'])).to(computing_device)
hidden = (hidden_state, cell_state)

model = Lstm(config['input_dim'], config['num_neurons'], config['num_layers'], hidden)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])

SyntaxError: invalid syntax (<ipython-input-4-4e03f794e53f>, line 1)

In [ ]:
#TODO : dataset and dataloader


In [58]:
for i in range(config['epochs']):
    #TODO : randomize the songs here
    
    
    #train
    model.train()
    for song in train_songs:
        checkpoint = torch.load(config['saved_path'])

        #set states to 0 at the beginning of each song
        hidden_state = torch.zeros((config['num_layers'], 1, config['input_dim'])).to(computing_device)
        cell_state = torch.zeros((config['num_layers'], 1, config['input_dim'])).to(computing_device)
        hidden = (hidden_state, cell_state)

        #restore the model's weights 
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

        #train here
        model.zero_grad()
    
        #TODO : fix this using the dataloader
        output, h = model(inputs, hidden)
        
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        optimizer.step()
        

        #save model
        torch.save({
                'model_state_dict': net.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, config['saved_path'])
    #validation
    model.eval()
    with torch.no_grad():
        for song in val_songs:
            
            #set states to 0 at the beginning of each song
            hidden_state = torch.zeros((config['num_layers'], 1, config['input_dim'])).to(computing_device)
            cell_state = torch.zeros((config['num_layers'], 1, config['input_dim'])).to(computing_device)
            hidden = (hidden_state, cell_state)
            
            
        

'<start>\nX:1\nT: La Montfarine\nZ:Transcrit et/ou corrig? par Michel BELLON - 2005-07-24\nZ:Pour toute observation mailto:galouvielle@free.fr\nM: 4/4\nL: 1/8\nQ:1/4=186\nFGF B=AG G=AG F2F FGF {F}F2E EFE|\n{E}E2D FGF B=AG G=AG {F}F2F FED C2G D2E|F3 {F}F/2 ED E3/2D/2|\nEC FE E2 DC|DC C2 GD2E|F3F/2F/2 DE FD|EC B,C D2 B,G|\ncB =A2 FG2E|F3F GF B=A|GG =AG F2 FF|GF F/2F3/2 EE FE|\nE/2E3/2 DF GF B=A|GG =AG F/2F3/2 FF|ED C2 GD2E|F3F/2F/2 ED E3/2D/2|\nEC FE E2 DC|DC C2 GD2E|F3F/2F/2 DE FD|EC B,C D2 B,G|\ncB =A2 FF G=A|B3\nG2_A G/2G3/2-|GF/2F/2 ED F2 EB,|CD EF GB AG|\nA3A GA F2|F/2F/2F EF D3/2B,/2 B,B,|CD EF GA GF|G3B/2B/2 AB G2|\nGG/2G/2 FG E2 EB/2B/2|AG BA GG AB|c3c/2c3/2d f2|ed eG c2 BB|\nAB dc BG AD|E3F GF BA|GG AG F2 FF|GF F/2F3/2 EE FE|\nE/2E3/2 DF GF BA|GG AG F/2F3/2 FF|ED C2 GD2E|F2- F/2\n<end>'